In [ ]:
import numpy as np
from math import sqrt

# Trabajo Práctico - Álgebra Lineal Computacional
## *Primer Cuatrimestre 2022*
#### Integrantes: 
1.  Domazet, Alejandro   (L.U. : 230/18)
2. Klimkowski, Victoria  (L.U. : 1390/21)
3. de Erausquin, Carla   (L.U. : 126/18)

## Ejercicio 1)

Deducir formulas para las coordenadas de L y D para el caso n = 3 tomando


$$A = \begin{pmatrix} a_{11} & a_{12} & a_{13} \\ a_{21} & a_{22} & a_{23} \\ a_{31} & a_{32} & a_{33} \end{pmatrix} $$

\\

$$L=\begin{pmatrix} 1 & 0 & 0 \\ l_{21} & 1 & 0 \\ l_{31} & l_{32} & 1 \end{pmatrix}$$

\\

$$D=\begin{pmatrix} d_1 & 0 & 0 \\ 0 & d_2 & 0 \\ 0 & 0 & d_3 \end{pmatrix}$$


Sabemos que

$$L^T = \begin{pmatrix} 1 & l_{12} & l_{12} \\ 0 & 1 & l_{23} \\ 0 & 0 & 1 \end{pmatrix}$$

Entonces
$$LD = \begin{pmatrix} d_1 & 0 & 0 \\ l_{21} * d_1 & d_2 & 0 \\ l_{31} * d_{1} & l_{32} * d_2 & d_3 \end{pmatrix} $$

Finalmente

$$A = LDL^T = \begin{pmatrix} d_1 & l_{21} * d_1 & l_{31} * d_1 \\ l_{21} * d_1 & l_{21}^2 * d_1 + d_2 & l_{31} * l_{21} * d_1 + l_{32} * d_2 \\ l_{31} * d_1 & l_{31} * l_{21} * d_1 + l_{32} * d_2 & l_{31}^2 * d_1 + l_{32}^2 * d_2 + d_3 \end{pmatrix} $$

De la factorización $$LDL^{T}$$ de A, podemos afirmar lo siguiente:


1.   $$a_{11}=d_1$$

2.   $$a_{21}=l_{21}*d_1 \iff l_{21}=\frac{a_{21}}{d_1}$$

1.   $$a_{31}=l_{31}*d_1 \iff l_{31}=\frac{a_{31}}{d_1}$$

2.   $$a_{22}=l_{21}^2*d_1+d_2 \iff d_2 = a_{22} - l_{21}^2*d_1$$

1.   $$a_{32}=l_{31}*l_{21}*d_1+l_{32}*d_2 \iff l_{32} = \frac{1}{d_2}*(a_{32} - l_{31}*l_{21}*d_1)$$

2.   $$a_{33}=l_{31}^2*d_1+l_{32}^2*d_2 + d_3 \iff d_{3} = a_{33} - (l_{31}^2*d_1 + l_{32}^2*d_2)$$









Finalmente, podemos deducir que



$$d_{jj} = a_{jj}-\sum_{k=1}^{j-1}l_{jk}*d_{kk}$$

$$l_{ij} = \frac{1}{d_{ij}}*(a_{ij}-\sum_{k=1}^{j-1}l_{ik}*l_{jk}*d_{kk})$$











## Ejercicio 2)

Para este ejercicio se implementó la desomposición *LU* de una matriz. Esto es, dada una matriz *A* se generó un código que retornara una matriz *U* triangular superior y una matriz *L* triangular inferior. Esta descomposición de la matriz pasada como parámetro tiene como objetivo la simplificada resolución de sistemas de ecuaciones, como se verá más adelante.  

In [ ]:
def descomposicion_LU(A):
    fils = np.shape(A)[0]
    cols = np.shape(A)[1]

    #Iniclizo L & U en ceros:
    L = np.zeros((fils, cols))  #Notar que, por defecto, inicializa en punto flotante
    U = np.zeros((fils, cols))

    #Como para L sé que necesito 1 en la diagonal, los escribo:
    for i in range(fils):
        L[i][i] = 1

    #Ahora puedo calcular el resto de los valores de L y U:

    for i in range(cols):
        if i == 0:
            U[0][0] = A[0][0]
            for j in range(1, cols):
                if U[0][0] == 0:
                    print("No se puede dividir por 0! No existe descomposición LU")
                    return np.identity(fils), A
                U[0][j] = A[0][j]
                L[j][0] = A[j][0] / U[0][0]
        else:
            for j in range(i, cols):  # U
                temp = 0
                for k in range(0, i):
                    temp = temp + L[i][k] * U[k][j]
                U[i][j] = A[i][j] - temp
            for j in range(i + 1, cols):  # L
                temp = 0
                for k in range(0, i):
                    temp = temp + L[j][k] * U[k][i]
                if U[i][i] == 0:
                    print("No se puede dividir por 0! No existe descomposición LU")
                    return np.identity(fils), A
                L[j][i] = (A[j][i] - temp) / U[i][i]
    return L, U


## Ejercicio 3)

Para este ejercicio se pedía implementar dos funciones para resolver sistemas de ecuaciones *Ly = b* y *Ux = y* suponiendo que las matrices *L* y *U* fueran triangulares inferior y superior respectivamente y que no tuvieran 0’s en la diagonal. 
Notamos que para resolver el primer problema podíamos emplear la siguiente fórmula cerrada: 

$$x_{[i]}=\frac{b_{[i]}-\sum_{j}^{n}U_{[i][j]}*x_{[j]}}{U_{[i][j]}}$$

donde *x* es el el vector cuya solución se desea conocer, *n* es el número de filas total de *U* e *i* es la iteración actual y *j* representa a los valores del vector *x* ya calculados previamente. Es por esto que para el cálculo de *x* se iteró de "atrás para adelante", empleando la función auxiliar *sumaAnteriores*. El comportamiento de *Lx = y* es análogo, solo que el rellenado es en el orden "natural" (esto es, de 0 a *n*). Notar que a la hora de implementar el código se tuvieron fuertemente en cuenta las hipótesis del enunciado por lo que no se contempló el escenario en el cual no se puede dividir por cero.  

### a)

In [ ]:
def sol_Lyx(L, x): #Lx=y
    fils = np.shape(L)[0]
    cols = np.shape(L)[1]

    y = np.zeros(cols) #Genero mi vector x

    if fils != np.shape(x)[0]: #Me aseguro de que filas b = filas U
        raise Exception("La cantidad de filas de L es distinta a la de x, no hay solución")

    for i in range(fils):
        y[i] = (x[i] - sumaPosteriores(L, y, i))/L[i][i] #Como ya asumimos que la matriz U es triangular superior,
                                                          # no corro el riesgo de dividir por 0
    return y

def sumaPosteriores(L, y, i):
    res = 0
    fils = np.shape(L)[0]

    for j in range(fils):
        res += L[i][j] * y[j]

    return res

### b)

In [ ]:
def sol_Uxb(U, b): #Ux=y
    fils = np.shape(U)[0]
    cols = np.shape(U)[1]

    x = np.zeros(cols) #Genero mi vector x

    if fils != np.shape(b)[0]: #Me aseguro de que filas b = filas U
        raise Exception("La cantidad de filas de U es distinta a la de b, no hay solución")

    for i in range(fils-1, -1, -1): #Voy de atrás para adelante
        x[i] = (b[i] - sumaAnteriores(U, x, i))/U[i][i] #Como ya asumimos que la matriz U es triangular superior,
                                                          # no corro el riesgo de dividir por 0
    return x

def sumaAnteriores(U, x, i):
    res = 0
    fils = np.shape(U)[0]

    for j in range(i, fils):
        res += U[i][j] * x[j]

    return res

## Ejercicio 4)

En este ejercicio se realizó un *testeo* de las funciones generadas en el ejercicio anterior. Para ello se usó la función *rand* para generar matrices cuadradas y vectores aleatorios. Luego se resolvieron los sistemas de ecuaciones generados con nuestro código y con las funciones aportadas por *linalg*.

In [ ]:
def testearEj3(filas):
    #Creamos A y b con números aleatorios:
    A = np.random.rand(filas, filas)
    b = np.random.rand(filas)


    print("La Matriz A es:\n{}".format(A))
    print("El vector b es:\n{}".format(b))
    
    #Descompongo A en L y en U con lo visto en el ejercicio 2:
    L, U = descomposicion_LU(A)
    print("La matriz U es:\n{}".format(U))
    print("La matriz L es:\n{}".format(L))

    #Resuelvo Ly = b:
    y = sol_Lyx(L, b)

    #Resuelvo Ux = y:
    res = sol_Uxb(U, y)

    error = (A @ res) - b

    norma1 = 0
    for i in range(len(error)):
      norma1 += abs(error[i])

    print(res)
    print(norma1)

    return res


In [ ]:
testearEj3(10)

La Matriz A es:
[[2.23313300e-01 9.41272782e-01 2.36803887e-01 5.02888040e-01
  2.30505623e-01 4.64382807e-01 5.83868508e-01 7.23908180e-01
  8.92672664e-01 4.74560873e-01]
 [7.17496085e-01 4.15183721e-01 3.12562321e-01 3.32251936e-01
  2.04171073e-02 6.79860620e-01 1.37883046e-01 7.24621470e-01
  1.50187883e-01 2.66873594e-01]
 [2.06901210e-01 9.87994761e-01 3.13698652e-01 1.64062045e-01
  5.91593971e-01 7.21045465e-01 7.01389896e-01 1.14860874e-01
  7.53281472e-07 5.26034433e-01]
 [2.87063180e-01 2.47580083e-01 5.53374501e-01 2.29999339e-02
  8.95061179e-01 5.62344079e-01 9.28009366e-01 3.41514627e-01
  9.87409851e-01 4.81456027e-01]
 [5.98853562e-01 3.47788246e-01 3.02198339e-01 5.60940877e-01
  5.85565664e-01 3.92870681e-01 2.20348516e-01 6.66489447e-01
  9.31457305e-01 9.05160300e-01]
 [1.87898547e-01 7.57326946e-01 3.86178828e-01 5.13892365e-01
  7.59196806e-01 8.61110034e-01 3.81483182e-01 9.09247718e-01
  9.02228362e-01 6.89407183e-01]
 [3.37644079e-01 4.75317858e-01 5.74748748

array([ 0.46409518,  0.73059973,  2.09499118, -0.34237052,  0.40667198,
       -1.19135396, -1.29406902,  0.21326694, -0.07895814,  0.51923365])

## Ejercicio 5)

In [ ]:
def cholesky(A):
    fils = np.shape(A)[0]
    L = np.identity(fils)
    D = np.zeros((fils, fils))

    for i in range(fils):
        D[i][i] = A[i][i] - suma_1(L, D, i)

        for j in range(i+1, fils):
            L[j][i] = (A[j][i] - suma_2(L, D, i, j)) / D[i][i]


    return L, D

def suma_1(L, D, i):
    res = 0

    for j in range(i):
        res += (L[i][j] ** 2) * D[j][j]

    return res

def suma_2(L, D, i, j):
    res = 0

    for k in range(i):
        res += (L[j][k] * L[i][k] * D[k][k])

    return res


## Ejercicio 6)
Para este ejercicio se tuvo fuertemente en cuenta que la matriz *D* en cuestión es diagonal. Luego, se observó que para cada valor del vector *x* podíamos realizar el siguiente cociente:
$$x_{[i]}=\frac{b_{[i]}}{d_{[i]}}$$

siempre vericando que el i-ésimo elemento de *d* sea distinto de cero:

In [ ]:
def sol_Dxb(D, b):
    filas = D.shape[0]
    x = np.zeros(filas)

    for i in range(filas):
        if(D[i][i] == 0):
            raise Exception("Hay un 0 en d! Entonces no hay solución")
            break
        x[i] = b[i]/D[i][i]

    return x

## Ejercicio 7)
Para este ejercicio, se pide justificar que $AA^T$ es simétrica y definida positiva. Para ello, partimos con la hipótesis en la cual A es una matriz invertible.

Ahora, $AA^T$ es simétrica pues $$(AA^T)^T = (A^T)^TA^T = AA^T$$

Por otro lado $$Q=A^T*A$$ es definida positiva $$ \iff x^TQ x>0$$ $$\forall x \neq 0$$

Demostración:

$$x^TQx = x^TA^TAx = (Ax)^TAx > 0$$ 

Pero $(Ax)^TAx$ es un producto interno

$$(Ax)^T(Ax) = ||Ax||^2 > 0$$ $$∀x \neq 0$$

## Ejercicio 8) 

Se probó la implementación realizada de *LDLt* para resolver un sistema *Qx = b* tomando una matriz A ∈ R
10×10 de números aleatorios, generada con funciones similares a las vistas para el **Ejercicio 4**. Para ello se descompuso la matriz generada con la función *sol_Qxb*
Luego en la función *ejercicio_8* se llamó a la anterior para comparar su resultado con el valor de *b* real, con la norma-1 del error *e = Qx - b*. Para ello se realizó la siguiente sumatoria:

$$e = \sum_{i=0}^{n}|(Q*x^t)_{[i]} - b_{[i]}|$$

In [ ]:
def sol_Qxb(n):
    A = np.random.randint( 100, size=(n, n))
    A_t = np.transpose(A)
    Q = A_t @ A
    b = np.random.randint( 100, size=(n))

    #Descompongo Q:
    L, D = cholesky(Q)

    #Ahora resuelvo el sistema:
    y = sol_Lyx(L, b)

    #Resuelvo Dz = y :
    z = sol_Dxb(D, y)

    #Ahora resuelvo L*y = b
    x = sol_Uxb(np.transpose(L), z)

    return x, b, Q

def ejercicio_8(n):

    #Primero pido x por el método de recién
    x, b, Q = sol_Qxb(n)

    #Ahora que tengo mi x y mi valor real, comparo:
    error = Q @ x - b
    suma = 0

    for i in range(len(error)):
        suma += abs(error[i])
        
    print("El valor real de b es:\n{} \n, haciendo Q*x es:\n {}\n porque el x obtenido fue \n{}\n y la norma 1 del error es:\n {}".format(b, Q@x, x, suma))

    return x, b, Q, suma

In [ ]:
ejercicio_8(10)

El valor real de b es:
[24 32 93 74 35 77 45 97  1 92] 
, haciendo Q*x es:
 [24. 32. 93. 74. 35. 77. 45. 97.  1. 92.]
 porque el x obtenido fue 
[-1.82747198 -0.77984894  0.6002708   1.4292539  -0.87407858 -2.82752382
  3.15362546  0.29824035 -2.28229726  3.42649811]
 y la norma 1 del error es:
 1.7462298274040222e-10


(array([-1.82747198, -0.77984894,  0.6002708 ,  1.4292539 , -0.87407858,
        -2.82752382,  3.15362546,  0.29824035, -2.28229726,  3.42649811]),
 array([24, 32, 93, 74, 35, 77, 45, 97,  1, 92]),
 array([[20827, 15838, 22238, 14936, 17824, 13952, 17485, 21486, 22058,
         17383],
        [15838, 27738, 28842, 17228, 22443, 17371, 16560, 27231, 24040,
         20991],
        [22238, 28842, 39874, 21833, 29022, 27688, 22003, 36180, 29545,
         28890],
        [14936, 17228, 21833, 24414, 20816, 20202, 16020, 30015, 23601,
         18244],
        [17824, 22443, 29022, 20816, 29911, 21935, 16856, 31714, 25865,
         25542],
        [13952, 17371, 27688, 20202, 21935, 31432, 22681, 35526, 25790,
         22884],
        [17485, 16560, 22003, 16020, 16856, 22681, 28221, 26016, 26421,
         14947],
        [21486, 27231, 36180, 30015, 31714, 35526, 26016, 50625, 35036,
         31219],
        [22058, 24040, 29545, 23601, 25865, 25790, 26421, 35036, 33681,
         25163],
 

## Ejercicio 9)

Calculamos LU de Q.

In [ ]:
def ejercicio_9(n):
  x, b, Q, norma1 = ejercicio_8(n)
  
  Lchol, Dchol = cholesky(Q)
  
  L, U = descomposicion_LU(Q)

  print("La descomposición de Cholesky es L= \n{}\n y D= \n{}\n".format(Lchol, Dchol))
  print("La descomposición de LU es L= \n{}\n y U=\n{}\n".format(L, U))
  print("La diferencia en cada casilla U\n {}\n".format(abs(Dchol@ np.transpose(L) - U)))
  print("La diferencia en cada casilla L\n {}\n".format(abs(Lchol - L )))

  return Lchol, Dchol, L, U

In [ ]:
ejercicio_9(10)

El valor real de b es:
[49  6 33 62 31 29 71 14 74 25] 
, haciendo Q*x es:
 [49.  6. 33. 62. 31. 29. 71. 14. 74. 25.]
 porque el x obtenido fue 
[-0.03671988 -0.15836187  0.06424281  0.05472462  0.11546283 -0.12313048
  0.04958547 -0.01265937  0.15055835 -0.04304211]
 y la norma 1 del error es:
 9.322320693172514e-12
La descomposición de Cholesky es L= 
[[ 1.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 1.10155174  1.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 1.14181434  0.57278027  1.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 1.05780002  0.25260966  0.13203112  1.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.71687632  0.43348645 -0.27887249 -0.78034431  1.          0.
   0.          0.          0.          0.        ]
 [ 0.79010192  0.40437771  0.25369535  0.38217289  0.005816

(array([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 1.10155174,  1.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 1.14181434,  0.57278027,  1.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 1.05780002,  0.25260966,  0.13203112,  1.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.71687632,  0.43348645, -0.27887249, -0.78034431,  1.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.79010192,  0.40437771,  0.25369535,  0.38217289,  0.00581668,
          1.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.92392801,  0.30512193, -0.16487653,  0.37824603,  0.08260694,
          0.37356481,  1.       

Podemos ver que la diferencia entre la $L$ de $LU$ y la $L$ de Cholesky es casi nula.
Lo mismo ocurre con la $U$ de $LU$ y la multiplicación de $D*L^T$.

En todas las casillas ocurre que el error es o bien 0 o bien de un orden tan bajo que es despreciable.


## Ejercicio 10)
Para la resolución de este ejercicio nos basamos fuertemente en lo realizado en el **Ejercicio 8**. La única diferencia es que se generó una nueva matriz, *W*, la cual era una permutación de columnas de la matriz *Q* como se observa más abajo:

In [ ]:
def ejercicio_10(n):

    #Primero pido x por el método de recién
    x, b, Q, sumaQ = ejercicio_8(n)

    #Armo la matriz de permutación:
    W = np.zeros((n, n))

    for i in range(0, n//2):
        W[:, i] = Q[:, n//2 + i]
        W[:, n//2 + i] = Q[:, i]


    #Descompongo W:
    L , U = descomposicion_LU(W)

    #Ahora resuelvo el sistema:

    #Ahora resuelvo L*y = b
    y = sol_Lyx(L, b)

    #Ahora resuelvo U*x = y
    x = sol_Uxb(U, y)

    #Ahora que tengo mi x y mi valor real, comparo:
    error = (W @ x) - b
    sumaW = 0

    for i in range(len(error)):
        sumaW += abs(error[i])

    print("El valor real de b es:\n{} \n, haciendo W*x es:\n {}\n porque el x obtenido fue \n{}\n y la norma 1 del error es:\n {}".format(b, W@x, x, sumaW))

    return Q, W, sumaQ, sumaW

In [ ]:
ejercicio_10(10)

El valor real de b es:
[27 96 45  1 69 22 36 67 80 54] 
, haciendo Q*x es:
 [27. 96. 45.  1. 69. 22. 36. 67. 80. 54.]
 porque el x obtenido fue 
[-0.0024056   0.02319157 -0.05390859 -0.02741671 -0.01274221 -0.02675204
  0.01317851  0.00420023  0.04122428  0.04773523]
 y la norma 1 del error es:
 3.410605131648481e-12
El valor real de b es:
[27 96 45  1 69 22 36 67 80 54] 
, haciendo W*x es:
 [27. 96. 45.  1. 69. 22. 36. 67. 80. 54.]
 porque el x obtenido fue 
[-0.02675204  0.01317851  0.00420023  0.04122428  0.04773523 -0.0024056
  0.02319157 -0.05390859 -0.02741671 -0.01274221]
 y la norma 1 del error es:
 4.774847184307873e-12


(array([[27659, 26185, 24491, 29370, 14155, 17701, 14305, 19348, 26924,
         18560],
        [26185, 33833, 30682, 32922, 23906, 23468, 16704, 28395, 30203,
         26792],
        [24491, 30682, 32737, 31043, 19895, 25873, 18167, 26400, 31358,
         27462],
        [29370, 32922, 31043, 40591, 23414, 24965, 20725, 29498, 33225,
         27108],
        [14155, 23906, 19895, 23414, 23916, 15755, 13166, 24168, 18444,
         19984],
        [17701, 23468, 25873, 24965, 15755, 29229, 14924, 26959, 25881,
         25252],
        [14305, 16704, 18167, 20725, 13166, 14924, 17446, 14611, 18639,
         15459],
        [19348, 28395, 26400, 29498, 24168, 26959, 14611, 35441, 29222,
         24511],
        [26924, 30203, 31358, 33225, 18444, 25881, 18639, 29222, 36219,
         23287],
        [18560, 26792, 27462, 27108, 19984, 25252, 15459, 24511, 23287,
         28584]]),
 array([[17701., 14305., 19348., 26924., 18560., 27659., 26185., 24491.,
         29370., 14155.],
        [

Se observó en esta primera corrida que la diferencia entre los errores no es significativa, sin embargo, el error de Q (definida positiva) es de un orden levemente menor. 

## Ejercicio 11)


In [ ]:
def ejercicio_11():

    Q, W, x, y = ejercicio_10(10)
    condQ = np.linalg.cond(Q, 1)
    condW = np.linalg.cond(W, 1)



    print("La condición 1 promedio para Q es:\n  {} \n  La condición 1 promedio para W es:\n {} \n ".format(condQ, condW))
    
    return 0

In [ ]:
ejercicio_11()

El valor real de b es:
[77 66 66 90  5 22 17 73 76 47] 
, haciendo Q*x es:
 [77. 66. 66. 90.  5. 22. 17. 73. 76. 47.]
 porque el x obtenido fue 
[-0.00963113  0.03991486  0.07114508  0.03642068 -0.04461629 -0.05060795
 -0.04327717 -0.03779445  0.05602174 -0.06485502]
 y la norma 1 del error es:
 4.092726157978177e-12
El valor real de b es:
[77 66 66 90  5 22 17 73 76 47] 
, haciendo W*x es:
 [77. 66. 66. 90.  5. 22. 17. 73. 76. 47.]
 porque el x obtenido fue 
[-0.05060795 -0.04327717 -0.03779445  0.05602174 -0.06485502 -0.00963113
  0.03991486  0.07114508  0.03642068 -0.04461629]
 y la norma 1 del error es:
 4.206412995699793e-12
La condición 1 promedio para Q es:
  8057.074369864692 
  La condición 1 promedio para W es:
 8057.074369864336 
 


0

Las condiciones son similares ya que la permutación de columnas de Q (con todas linealmente independientes) no afecta qué tan lejos estará W (la permutada) de una matriz singular.

Se obtuvo que la condición de ambas matrices es casi idéntica.


## Ejercicio 12)

In [ ]:
def ejercicio_12(n):

    sumaQ = 0
    sumaW = 0

    for i in range(100):
        Q, W, q, w = ejercicio_10(n)
        sumaQ += q
        sumaW += w

    promedioQ = sumaQ/100
    promedioW = sumaW/100

    print("El promedio de errores de Q es: {} y el de W es {}".format(promedioQ, promedioW))
    return promedioQ, promedioW



El promedio de errores de Q es levemente menor al de W ya que Q es definida positiva.
Coincide parcialmente con el resultado que esperabamos ya que como Q estaba tiene propiedades más fuertes, creíamos que la diferencia iba a ser aún mayor.
Sin embargo, es coherente con los resultados que obtuvimos en los otros puntos (que las condiciones de ambas matrices son similares, que los errores similares, que los métodos de Cholesky y LU apenas tienen diferencia alguna para Q, etcétera).

In [ ]:
ejercicio_12(100)

Se truncaron las últimas líneas 5000 del resultado de transmisión.
   57.18214403  179.36758757 -147.79389082 -404.46436359  252.38776624
 -674.44330702  200.56137959  -55.75172729  162.53070232   70.84640632
 -391.08076868  217.93536428 -429.64603317 -670.6985918  -688.38423122
 -129.04848596  -58.1586036  -103.9099823   -82.16905018 -182.45693496
  378.65970724 -282.59688228  364.72323158  177.4306796  -333.71318552
   29.44114261  467.95398077   30.7607975   -36.48071728  -22.69740923
 -248.3169562   190.37745443 -159.61720776  549.15422297 -159.36531476
 -170.59904766 -105.74861537 -219.2766666  -265.94989196  -46.42905514
  230.86503623  286.0369264   262.57918194 -275.38344368  391.2086463
   94.54162167  178.21412426 -189.46202843  119.70436278   80.6276638
  342.5927924   175.18320552 -221.58990207   42.38293413 -187.01561514
 -617.28494175 -602.72307667  -85.46813109 -584.01601317  358.14688456
  371.28809172 -394.01340819  268.14287371  345.36380722 -169.20434065
  -48.740150

(5.911174748689518e-06, 4.8585724266558826e-05)